# Navegación en HTML y Extracción de Tablas con BeautifulSoup

## 1. Introducción a la Estructura HTML

Cuando cargamos una página web en un navegador, lo que realmente obtenemos es un documento **HTML**, que es el código fuente de la página.

### Cómo inspeccionar el HTML de una página web:

1. **Hacer clic derecho en la página → 'Inspeccionar'** (en Chrome o Firefox).

2. **Ver el código fuente** con `Ctrl + U` o `Cmd + U` en Mac.

3. **Escribir en la barra del navegador**: `view-source:https://en.wikipedia.org/wiki/Passengers_of_the_Titanic`.

## 2. Ubicar la Tabla en el HTML

Si inspeccionamos la página de Wikipedia, veremos que los datos de los pasajeros están organizados en **tablas** dentro de elementos `<table>` con la clase `'wikitable sortable'`.

### Ejemplo de estructura HTML de la tabla:

In [ ]:
<table class='wikitable sortable'>
    <tr>
        <th>Name</th>
        <th>Age</th>
        <th>Class</th>
    </tr>
    <tr style='background:#9bddff;'>
        <td>John Smith</td>
        <td>30</td>
        <td>1</td>
    </tr>
</table>

## 3. Cómo Extraer la Tabla con `BeautifulSoup`

Usamos **BeautifulSoup** para encontrar todas las tablas en la página y extraer su contenido.

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL de Wikipedia
wikipage = 'https://en.wikipedia.org/wiki/Passengers_of_the_Titanic'

# Hacer la petición HTTP
page = requests.get(wikipage)

# Parsear el contenido HTML
soup = BeautifulSoup(page.content, 'html.parser')

# Buscar todas las tablas con la clase específica
wikitables = soup.find_all('table', {'class': 'wikitable sortable'})

print(f'Se encontraron {len(wikitables)} tablas en la página.')

In [ ]:
for i, table in enumerate(wikitables):
    # Intentar obtener el título de la tabla si hay una etiqueta <caption>
    title_tag = table.find('caption')
    title = title_tag.get_text(strip=True) if title_tag else f'Tabla {i+1} (Sin título)'

    # Contar las filas de la tabla (sin incluir la fila de encabezados)
    rows = table.find_all('tr')
    num_rows = len(rows) - 1  # Restamos 1 para no contar el encabezado

    # Obtener los nombres de las columnas (th)
    headers = [header.get_text(strip=True) for header in table.find_all('th')]

    print(f'{title}')
    print(f'Número de filas: {num_rows}')
    print(f'Encabezados: {headers}\n')

## 4. Identificar a los Fallecidos en la Tabla (Basado en el Color)

Sabemos que los pasajeros fallecidos tienen `style='background:#9bddff;'`. Entonces, podemos filtrar las filas (`<tr>`) que contienen este atributo.

In [ ]:
for i, table in enumerate(wikitables):
    print(f'\nTabla {i+1}:')

    # Extraer todas las filas que tienen el color de fallecidos
    dead_rows = table.find_all('tr', style='background:#9bddff;')

    # Extraer nombres de los fallecidos
    for row in dead_rows:
        cells = row.find_all('td')  # Obtener celdas
        if len(cells) > 1:
            name = cells[0].get_text(strip=True)
            print(f'Fallecido: {name}')

## 5. Construcción de un DataFrame con Pandas

Podemos extraer la información y almacenarla en un **DataFrame de Pandas** para un análisis más profundo.

### Código:

In [ ]:
import pandas as pd

# Lista para almacenar pasajeros
passengers = []

# Iteramos sobre cada tabla encontrada
for i, table in enumerate(wikitables):
    passenger_class = i + 1  # Primera tabla es 1ª clase, segunda es 2ª, etc.

    # Filtrar filas según el color de fondo
    dead_rows = table.find_all('tr', style='background:#9bddff;')
    all_rows = table.find_all('tr')

    # Identificar filas de personas vivas (sin estilo)
    living_rows = [row for row in all_rows if row not in dead_rows]

    # Procesar filas
    for rows, survived in [(living_rows, 1), (dead_rows, 0)]:
        for row in rows:
            cells = row.find_all('td')
            if len(cells) < 2:
                continue  # Saltamos filas sin datos

            name = cells[0].get_text(strip=True)
            age = cells[1].get_text(strip=True) if cells[1].get_text(strip=True) else None

            passengers.append({'name': name, 'class': passenger_class, 'age': age, 'survived': survived})

# Convertir a DataFrame
df_passengers = pd.DataFrame(passengers)

# Mostrar los primeros datos
print(df_passengers.head())